In [12]:
TAM_IMG = 200
BATCH_SIZE_LOADER = 32
COLOR_MODE = 'rgb'

In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,          # Escalado de píxeles al rango [0, 1]
)

val_datagen = ImageDataGenerator(rescale=1.0/255.0)

test_datagen = ImageDataGenerator(rescale=1.0/255.0)

In [14]:
# Directorios
train_dir = "../Celebrity_Faces_Dataset_Duplicada_SPLITTED/train"
val_dir = "../Celebrity_Faces_Dataset_Duplicada_SPLITTED/val"

COLOR_MODE = 'rgb'
# Crear generadores de datos
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=(TAM_IMG, TAM_IMG),
    batch_size=BATCH_SIZE_LOADER,
    color_mode=COLOR_MODE,
    class_mode='categorical'
)

val_data = val_datagen.flow_from_directory(
    val_dir,
    target_size=(TAM_IMG, TAM_IMG),
    batch_size=BATCH_SIZE_LOADER,
    color_mode=COLOR_MODE,
    class_mode='categorical'
)

test_dir = "../Celebrity_Faces_Dataset_Duplicada_SPLITTED/test"

test_data = test_datagen.flow_from_directory(
    test_dir,
    target_size=(TAM_IMG, TAM_IMG),
    batch_size=BATCH_SIZE_LOADER,
    class_mode='categorical',
    shuffle=False
)


Found 1197 images belonging to 17 classes.
Found 341 images belonging to 17 classes.
Found 173 images belonging to 17 classes.


### Modelo

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization, Input
from tensorflow.keras.regularizers import l2

In [16]:
input_shape = (TAM_IMG, TAM_IMG, 3)
neuronas_salida = len(train_data.class_indices)
filtros_conv = 32
neuronas_capa_densa = 128
kernel_size = (3, 3)
padding_tipo = 'valid'

# Modelo con 4 capas convolucionales, 3 de ellas con el mismo número de filtros
model_2 = Sequential([
    Input(shape=input_shape),

    Conv2D(filtros_conv, kernel_size, activation='relu', padding=padding_tipo),
    MaxPooling2D((2, 2)),

    Conv2D(filtros_conv*2, kernel_size, activation='relu', padding=padding_tipo),
    MaxPooling2D((2, 2)),

    Conv2D(filtros_conv*4, kernel_size, activation='relu', padding=padding_tipo),
    MaxPooling2D((2, 2)),

    Conv2D(filtros_conv*8, kernel_size, activation='relu', padding=padding_tipo),
    MaxPooling2D((2, 2)),
    
    Flatten(),
    Dense(neuronas_capa_densa, activation='relu'),
    Dropout(0.5),
    Dense(neuronas_salida, activation='softmax')
])

model_2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model_2.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 198, 198, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 99, 99, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 97, 97, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 48, 48, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 46, 46, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 23, 23, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 21, 21, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 10, 10, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25600)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │     3,276,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 17)             │         2,193 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,667,537 (13.99 MB)

 Trainable params: 3,667,537 (13.99 MB)

 Non-trainable params: 0 (0.00 B)

In [17]:
model_2.fit(train_data, validation_data=val_data, epochs=20, batch_size=32)

Epoch 1/20


/home/david/miniconda3/envs/tf_wsl/lib/python3.9/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-11-23 18:50:17.212784: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[32,32,198,198]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,3,200,200]{3,2,1,0}, f32[32,3,3,3]{3,2,1,0}, f32[32]{0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]

23/38 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step - accuracy: 0.0743 - loss: 2.9336

2024-11-23 18:50:25.556366: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[13,32,198,198]{3,2,1,0}, u8[0]{0}) custom-call(f32[13,3,200,200]{3,2,1,0}, f32[32,3,3,3]{3,2,1,0}, f32[32]{0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]}
2024-11-23 18:50:25.678552: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[13,64,97,97]{3,2,1,0}, u8[0]{0}) custom-call(f32[13,32,99,99]{3,2,1,0}, f32[64,32,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_con

38/38 ━━━━━━━━━━━━━━━━━━━━ 0s 167ms/step - accuracy: 0.0724 - loss: 2.9085

2024-11-23 18:50:30.529306: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[32,32,198,198]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,3,200,200]{3,2,1,0}, f32[32,3,3,3]{3,2,1,0}, f32[32]{0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kRelu","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]}
2024-11-23 18:50:30.762263: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[32,64,97,97]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,32,99,99]{3,2,1,0}, f32[64,32,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_con

38/38 ━━━━━━━━━━━━━━━━━━━━ 19s 282ms/step - accuracy: 0.0723 - loss: 2.9073 - val_accuracy: 0.0645 - val_loss: 2.8330
Epoch 2/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 3s 78ms/step - accuracy: 0.0779 - loss: 2.8336 - val_accuracy: 0.0645 - val_loss: 2.8330
Epoch 3/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 3s 79ms/step - accuracy: 0.0699 - loss: 2.8325 - val_accuracy: 0.1026 - val_loss: 2.8268
Epoch 4/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 3s 77ms/step - accuracy: 0.0653 - loss: 2.8213 - val_accuracy: 0.1232 - val_loss: 2.7148
Epoch 5/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 3s 81ms/step - accuracy: 0.1119 - loss: 2.6823 - val_accuracy: 0.1613 - val_loss: 2.6143
Epoch 6/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 5s 139ms/step - accuracy: 0.1770 - loss: 2.5325 - val_accuracy: 0.2522 - val_loss: 2.4151
Epoch 7/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 3s 81ms/step - accuracy: 0.2589 - loss: 2.2900 - val_accuracy: 0.2522 - val_loss: 2.3464
Epoch 8/20
38/38 ━━━━━━━━━━━━━━━━━━━━ 3s 84ms/step - accuracy: 0.3136 - loss: 2.1456 - val_accuracy: 0.2874 - val_loss:

### Resultados

In [18]:
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import numpy as np

In [19]:
y_pred = model_2.predict(test_data)
y_pred_classes = np.argmax(y_pred, axis=1)

y_true = test_data.classes

acc = accuracy_score(y_true, y_pred_classes)
test_loss, test_acc = model_2.evaluate(test_data)

print(f"Accuracy sklearn (accuracy_score): {acc:.4f}")
print(f"Test Accuracy Keras (model.evaluate): {test_acc:.4f}")

5/6 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step

2024-11-23 18:52:16.410335: I external/local_xla/xla/service/gpu/autotuning/conv_algorithm_picker.cc:557] Omitted potentially buggy algorithm eng14{k25=0} for conv (f32[13,32,198,198]{3,2,1,0}, u8[0]{0}) custom-call(f32[13,3,200,200]{3,2,1,0}, f32[32,3,3,3]{3,2,1,0}, f32[32]{0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kRelu","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0","wait_on_operation_queues":[]}
2024-11-23 18:52:17.511224: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng18{k2=1,k4=2,k5=1,k6=0,k7=0} for conv (f32[13,64,97,97]{3,2,1,0}, u8[0]{0}) custom-call(f32[13,32,99,99]{3,2,1,0}, f32[64,32,3,3]{3,2,1,0}, f32[64]{0}), window={size=3x3}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convBiasActivationForward", backend_config={"cudnn_conv_

6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 640ms/step
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 95ms/step - accuracy: 0.2465 - loss: 3.7172
Accuracy sklearn (accuracy_score): 0.2948
Test Accuracy Keras (model.evaluate): 0.2948


Solo agregando más capas y manrteniendo el aumento de filtros el modelo mejore un poco  
modelo_1 = 0.1676  
modelo_2 = 0.2948  

In [21]:
# guardamos el modelo
model_2.save('../modelos/modelo_1-2_MasCapas_acc_0-29.keras')